## Naming conventions 

* _single_leading_underscore: weak "internal use" indicator
 * e.g., from M import * does not import objects whose names start with an underscore
* single_trailing\_underscore_: use to avoid conflicts with keywords
* \__double_leading_underscore: used to name a class attribute
 * invokes name mangling (inside class Class \__par is accessible, outside it becomes _Class__par)
* \__double_leading_and_trailing_underscore\__: "magic" objects or attributes such as \__init\__: should be never invented by user

In [10]:
%reset -f
class A(object): __id = "id of class A" # not part of A's API
class B(A):      __id = "id of class B" # not part of B's API, the name is the same for coincidence
                                        # thanks to name mangling, __id is not overwritten 

a = A()
b = B()
print('__id' in dir(a))    # False __id is hidden
print('_A__id' in dir(a))  # True  __id can be exposed through name mangling

print(a._A__id) # relative to A
print(b._B__id) # relative to B

False
True
id of class A
id of class B


## Introduction to decorators 

* functions in python (as we have seen)
 * can be assigned to names
 * can be defined inside other functions
 * can be passed as parameters to other functions
 * can return other functions
* decorators are wrappers that modify the behavior of the code (e.g., of a function)
 * i.e., decorators target an object and increase its original functionality, so "decorating" it 
 * a decorator is passed the original object being defined and returns a modified object 

```python
def myFun():  
    '''whatever behavior'''
    pass
def decoration():  
    '''whatever decoration'''
    pass
myFun = decoration(myFun)

```

#### the "syntactic sugar" to make decorators cleaner to read, python use the keyword @  

```python
def decoration():  
    '''whatever decoration'''
    pass
@decoration
def myFun():  
    '''whatever behavior'''
    pass    
```

#### multiple decorations can be chained 

In [187]:
%reset -f
def who_just_arrived(): return "jacopo"   # this is a function
def greet(whoever_I_want_to_greet):       # functions can be passed as parameters
    def hi(): return "hi "                # functions can be defined inside other functions
    return hi()+whoever_I_want_to_greet() # functions can return other functions           

say_hi_to_ = greet(who_just_arrived)      # functions can be assigned to names

print(say_hi_to_)

hi jacopo


In [19]:
%reset -f
def decorate(func):         # decoration inputs the function!
    def func_wrapper(name): # wrappers inputs the function's parameter(s)
        # check http://ascii-table.com/ansi-escape-sequences.php
        return   "\033[1m{0}\033[0m".format(func(name))
        # equals "\033[1m"+func(name)+"\033[0m" 
    return func_wrapper

def greet(name):
    return "hi {0}".format(name)

my_greet = decorate(greet)
print(greet("jacopo"))
print(my_greet("jacopo"))

hi jacopo
hi jacopo


In [13]:
%reset -f
def decorate(func):
    def func_wrapper(name):
        # check http://ascii-table.com/ansi-escape-sequences.php
        return "\033[1m{0}\033[0m".format(func(name))
    return func_wrapper

@decorate  
def greet(name): # -> greet = decorate(greet)
    return "hi {0}".format(name)

print(greet("jacopo"))

hi jacopo


In [225]:
%reset -f
def decorate_red(func): # makes greet red
    def func_wrapper(name):
        return "\033[91m{0}\033[0m".format(func(name))
    return func_wrapper

def decorate_bold(func): # makes greet bold
    def func_wrapper(name):
        # check http://ascii-table.com/ansi-escape-sequences.php
        return "\033[1m{0}\033[0m".format(func(name))
    return func_wrapper

@decorate_bold
@decorate_red
def greet(name): # -> greet = decorate_bold(decorate_red(greet))
    return "hi {0}".format(name)

print(greet("jacopo"))

hi jacopo


In [17]:
%reset -f
def formatting(how_to_format):
    def etiquette_decorator(func):
        def func_wrapper(name): 
            # check http://ascii-table.com/ansi-escape-sequences.php
            if how_to_format=='RED': 
                return   "\033[91m{0}\033[0m".format(func(name))                
            if how_to_format=='BOLD': 
                return "\033[1m{0}\033[0m".format(func(name))
            if how_to_format=='UNDERLINE': 
                return "\033[4m{0}\033[0m".format(func(name))            
        return func_wrapper
    return etiquette_decorator

@formatting("RED")        # you can pass arguments to decorators functions
@formatting("BOLD")
@formatting("UNDERLINE")
def greet(name):
    return "hi {0}".format(name)

print(greet('jacopo'))

hi jacopo


## Data encapsulation

* many object-oriented languages stress to have getter, setter and deleter methods to access private attributes
* this is called data encapsulation 
 * it does not hide attributes (nothing in python can do that)
* code loses readability: why should data be encapsulated?
 * to isolate code changes that depends on a specific attribute's value


In [61]:
%reset -f 
class Fruit(object):  # implementation using setter,    
                      #                      getter, 
                      #                      deleter
    def __init__(self,name): 
        self._set_name(name)
    def _set_name(self,name):     # setter
        self._name = name
    def _get_name(self):          # getter
        return self._name
    def _del_name(self):          # deleter
        del self._colour                        
    def __str__(self): 
        return "this fruit is an "+self._name

print(Fruit('apple'))

this fruit is an apple


##  Using property() to encapsulate data

* property(getter,setter\[,deleter,docstring\]): 
 * built-in function 
 * proxy any attribute access to getter, setter, deleter and docstring
* properties can embed customized actions when they are accessed

In [20]:
%reset -f 
class Fruit(object):
    _name=''
    def __init__(self,name): 
        self._set_name(name)
    def _set_name(self,name):        
        if self._name == '': 
            self._name = name
        else: 
            raise TypeError('Fruit name cannot be changed once set')
    def _get_name(self):          
        return self._name
    def _del_name(self):          
        del self._name
    def __str__(self): 
        return "this fruit is an "+self._name
        
    # assign properties
    name = property(_get_name,\
                    _set_name,\
                    _del_name,\
                    'name is property of Fruit')
    

f = Fruit('apple')
print(f)
try: 
    f.name = 'pear'     
except TypeError as err: 
    print(' -> err: ', err)
print(f.name)

this fruit is an apple
 -> err:  Fruit name cannot be changed once set
apple


## Property decorations

* you can use decorators to invoke property() 
 * you mark a function instead of call property() at the end of the class definition
 * you can avoid underscore prefixes 
 * (more readable)
* @property decorates the getter of the attribute \$foo
* @\$foo.setter decorates the setter
* @\$foo.deleter decorates the deleter

In [22]:
%reset -f 
class Fruit(object): # re-implementation using property decorators
    def __init__(self,name=None): 
        self._name = name
    @property    
    def name(self):     
        '''name is property of Fruit'''
        return self._name
    @name.setter
    def name(self,name):
        if not self._name: 
            self._name = name
        else: 
            raise TypeError('Fruit name cannot be changed once set')        
    @name.deleter
    def name(self):          
        del self._name
    def __str__(self): 
        return "this fruit is an "+self._name        

f = Fruit('apple')
print(f)
try: 
    f.name = 'pear'
except TypeError as err: 
    print(' -> err: ', err)
print(f.name)

this fruit is an apple
 -> err:  Fruit name cannot be changed once set
apple


## Why properties are useful

* methods should implement an action 
 * (commonly named with verbs)
* data attributes do not not implement an action 
 * (commonly named with a noun)
* properties differs from data attributes because...
 * they can embed customized actions when they are accessed 

In [26]:
%reset -f
class Values(list):     
    _positive=[]
    def __init__(self,data=None): 
        self._data    = data   
        self.positive = data # calls the setter
    @property
    def positive(self): 
        print("getting self._positive")
        return self._positive
    @positive.setter
    def positive(self,data): 
        print("setting self._positive")
        if not self._positive:  # prevents the use of this setter outside of this class
            [self._positive.append(d) for d in data if d>0]
        
v = Values([-1,1,-2,2,-3,3] )  
for i in range(3):
    print(v.positive)  # calls the setter only the first time, then only the getter

setting self._positive
getting self._positive
[1, 2, 3]
getting self._positive
[1, 2, 3]
getting self._positive
[1, 2, 3]


## Inheritance 

* a new class can inherit attributes from one or more "base" classes
 * this new class is called derived class
 * the base class must be in a scope containing the derived class
* if a requested attribute is not found in the class, the search proceeds to look in the base class   
 * this rule is applied recursively if base classe is derived itself
* a derived class can override methods of a base class
 * a method of a base class may end up calling a method of a derived class
 * this behaviour is equivalent to C++ virtual classes
 * the overriden base class method can be invoked explicitly (baseClass.methosName(self,args)) 
* sometimes overriden methods have a lot in commons with the relative base definition
 * super() is a built-in function that can be used to extend the base class method

#### isinstance(inst,class): True if inst is an instance of of class or derived 
#### issubclass(subClass,class): True if subClass derived from class 



In [2]:
%reset -f
class A(object):
    def f(self): return self.f0() # calls A.f0()
    def f0(self): return "f0 is a method of class A"
class B(A): 
    def f0(self): return "f0 is a method of class B"

a = A()    
b = B()
print(a.f()) 
print(b.f()) # b.f()->a.f()->(self).f0() == b.f0()
             # b inherits b from class A and calls A.f0()
             # however, A.f0() was overriden by B.f0()! 

f0 is a method of class A
f0 is a method of class B


In [7]:
%reset -f
class A(object):
    def f(self): pass
class B(A):       # inherits from A
    pass
class C(object):  # does not inherit from A
    pass
class D(B):       # inherits from B, which inherits from A
    pass

print(issubclass(B,A)) # True
print(issubclass(C,A)) # False
print(issubclass(D,A)) # True



True
False
True


In [8]:
a = A()
b = B()   # B inherits from A

print(isinstance(b,B)) # True 
print(isinstance(b,A)) # True 
print(isinstance(a,B)) # False 

True
True
False


## HAS A != IS A

* a class HAS attributes that define a specific behavior 
* DO NOT USE INHERITANCE JUST TO REUSE IMPLEMENTATION OF A METHOD OR AN ATTRIBUTE
 * conceptually Inheritance implements IS A 
 * Inheritance should never be used to implement HAS A
 * the derived class IS the same kind of the base class with additional features 

## Multiple inheritance

* a class can be derived by multiple base class
* search for attributes is depth-first, left-to-right 
 * Method Resolution Order algorithm 
 * base class should be accessed only once, otherwise TypeError is raised
* type(obj).mro(): returns the order in which the methods accessed by obj are searched 

In [36]:
%reset -f
class C(object): 
    def f_c(self): return "C has f_c"
class B(object): 
    def f_b(self): return "B has f_b"
class A(B,C): 
    def __str__(self): return "A IS both B and C"
    pass

a = A() 
print(a)
print(a.f_b()) # inherited from B
print(a.f_c()) # inherited from C

A IS both B and C
B has f_b
C has f_c


## super() and Method Resolution Order  

* super() means super(NextBaseClass,self)
 * MRO rules what means "next" 
 * diamond structure: what is method order?
```txt
|               A.f()
|              / \
|             /   \
|            /     \
|           B.f()   C.f()
|            \     / 
|             \   /  
|              \ /   
|               D.f()
```

In [64]:
%reset -f
class D(object): 
    def f(self): return "D"
class C(D): 
    def f(self): return "C"    
class B(D): 
    def f(self): return "B"
class A(B,C):    pass
class A2(C,B):   pass


a = A()
print(a.f()) # B or C?  B
print(A.mro())

a2 = A2()
print(a2.f()) # B or C? C 
print(A2.mro())


B
[<class '__main__.A'>, <class '__main__.B'>, <class '__main__.C'>, <class '__main__.D'>, <class 'object'>]
C
[<class '__main__.A2'>, <class '__main__.C'>, <class '__main__.B'>, <class '__main__.D'>, <class 'object'>]


In [61]:
b = B()
for i in [a,a2,b]:
    print(super(B,i).f()) # gets the MRO for object i, 
                          # call the f() for the class next to the MRO with respect to B

C
D
D


## Polymorphism 

* polymorphism is the provision of a single interface to entities of different types: 
 * subtyping: supported with inheritance
 * parametric polymorphism: supported with generic programming (e.g., templates)  
 * ad hoc polymorphism: supported in many languages using function overloading 
* implementation aspects
 * the implementation is static if selected at compile time (e.g. C++ templates, macros or function overloading) 
 * the implementation is dynamic if selected at run time (e.g. virtual functions, abstract base classes)
 
## Polymorphism in python

* subtyping (via inheritance)
* duck typing
* abstract base classes  (optional)
 

## Subtyping and duck typing

* a subtype is a datatype that is related to another datatype, i.e. the supertype, i.e. the interface
 * multiple subtype can refere to the same interface
 * polymorphic subtyping is a good reason to use inheritance 
* duck typing mimics subtyping without implementing inheritance
 * "If it walks like a duck and it quacks like a duck, then it must be a duck."

#### in normal programming style, suitability is assumed to be determined by an object's type only
#### in duck typing's style, suitability is determined by the interface (i.e., the presence of certain methods and properties)  


In [2]:
%reset -f
class Animal(object):  # interface class
    def __init__(self): pass
    def call(self): return self.animalSound
class Tiger(Animal):
    animalSound="roarrrrs" 
class Dog(Animal):
    animalSound="whoooofs"
class Bird(Animal):
    animalSound="tweets"  

def I_hear_(something):
    print('I hear... a '+\
          something.__class__.__name__,\
          'which',something.call())     
al=[]
al.append(Tiger())
al.append(Dog())
al.append(Bird())

for this_animal in al:
    # the interface method is invoked dynamically
    I_hear_(this_animal) 

I hear... a Tiger which roarrrrs
I hear... a Dog which whoooofs
I hear... a Bird which tweets


In [1]:
%reset -f
class Animal(object):  # interface class
    def __init__(self): pass
    def call(self): return self.animalSound
    
# duck typing example: 
# Duck is not subclass of Animal, but has the same interface
class Duck(object): 
    animalSound="definitely quacks like a duck"
    def __init__(self): pass
    def call(self): return self.animalSound    
    
def I_hear_(something):
    print('I hear... a '+\
          something.__class__.__name__,\
          'which',something.call()) 
    
I_hear_(Duck())

I hear... a Duck which definitely quacks like a duck


In [78]:
%reset -f
class multiple_of_(object):                        # I am duck-typing the class Container 
    def __init__(self,number): self.number=number  # constructor to indroduce an attribute number
    def __contains__(self,x):                      # method that defines which number belongs to this class
        if x<self.number: return False             # by definition, multiple of x should not be smaller than x 
        if x%self.number==0: return True           # if multiple of number, then it belongs to this class
        else: return False                         # otherwise it does not
    def __str__(self): 
        return 'is multiple of '+str(self.number)

m10 = multiple_of_(10)
m9 = multiple_of_(9)
m8 = multiple_of_(8)
m7 = multiple_of_(7)
for i in range(31):
    for j in [m7,m8,m9,m10]:             
        if i in j:                
            print(i,j)

7 is multiple of 7
8 is multiple of 8
9 is multiple of 9
10 is multiple of 10
14 is multiple of 7
16 is multiple of 8
18 is multiple of 9
20 is multiple of 10
21 is multiple of 7
24 is multiple of 8
27 is multiple of 9
28 is multiple of 7
30 is multiple of 10


### removed abc from this lecture

## Duck typing limits and abstract base classes (ABCs) 


* duck typing does not verify whether the class satisfies your requirements or not
 * the concept of abstract classes extends duck typing
 * an abstract class defines a set of attributes that a class must implement (i.e., limits the duck-type flexibility)
* collections module contains many ABCs implementations
 * e.g., the Container class
 * to duck-type an ABC some methods must be defines: check this using \__abstractmethods\__ 
* abc module provides the infrastructure to define new ABC classes
 * uses decorators

## How to define a context manager class  


* a context manager class must define two methods: 
 * \_\_enter\_\_(self): enter the runtime context 
 * \_\_exit\_\_(self, exc_type, exc_value, traceback): exit the runtime context
* useful to embed a specific protocol 

In [35]:
%reset -f 
class myErr(Exception): pass

class CManager(object):
    def __enter__(self):
        print('__enter__')
        return self    
    def __exit__(self, type, value, traceback):
        print('__exit__:', type, value)
        return True # if False no exception is considered

print('# with no interrupt')
with CManager() as c:
    print('doing something with ', c)
print('done something\n')    
    
print('# with interrupt')
with CManager() as c:
    print('doing something with ', c)
    raise myErr('err msg')
    print('unreachable')
print('done something')




# with no interrupt
__enter__
doing something with  <__main__.CManager object at 0x7ffb0c46b470>
__exit__: None None
done something

# with interrupt
__enter__
doing something with  <__main__.CManager object at 0x7ffb0c46b2b0>
__exit__: <class '__main__.myErr'> err msg
done something


## contextlib module

* decorator @contextmanager: combined with yield, can be used to define context manager even without defining a class
* ContextDecorator: base class to enable a context manager to be used as a decorator


In [131]:
%reset -f
from contextlib import contextmanager

@contextmanager
def tag(name): 
    print("<{}> ".format(name),end="")
    yield
    print(" </{}>".format(name),end="")
    
with tag("thisTag"): 
    print("something",end="")    

<thisTag> something </thisTag>

In [4]:
%reset -f
from contextlib import ContextDecorator
import time
class Timer(object):
    def __enter__(self):
        self.__t1 = time.time()
        return self
    def __exit__(self,*exc):
        self.__t2 = time.time()
        print(1000*(self.__t2-self.__t1),'ms')
        return False
    
with Timer():
    for i in range(10):         
        with Timer(): 
            print('',end='')
    print("whole time: ")

0.18143653869628906 ms
0.028133392333984375 ms
0.3943443298339844 ms
0.10967254638671875 ms
0.10013580322265625 ms
0.09894371032714844 ms
0.10061264038085938 ms
0.09894371032714844 ms
0.09870529174804688 ms
0.09918212890625 ms
whole time: 
3.642559051513672 ms


In [7]:
%reset -f
from contextlib import ContextDecorator
import time
class Timer(ContextDecorator):
    def __enter__(self):
        self.__t1 = time.time()
        return self
    def __exit__(self,*exc):
        self.__t2 = time.time()
        print(1000*(self.__t2-self.__t1),'ms')
        return False
@Timer()
def fun(): 
    print('',end='')
@Timer()
def ten_times_fun():
    for i in range(10): 
        fun()
    print("whole time: ")
    
ten_times_fun()        
    

0.11539459228515625 ms
0.030517578125 ms
0.2498626708984375 ms
0.15091896057128906 ms
0.12159347534179688 ms
0.12111663818359375 ms
0.12254714965820312 ms
0.12087821960449219 ms
0.12087821960449219 ms
0.12111663818359375 ms
whole time: 
4.659891128540039 ms


## Iterable, sequences and iterators

* an iterable is an object with \_\_iter\_\_() method
 * \_\_iter\_\_() normally return the class itself
* a sequence is an iterable with \_\_getitem\_\_() and \_\_len\_\_() methods 
 * e.g., list, str, tuple are sequences 
* an iterator is an iterable with \_\_next\_\_() method
 * \_\_next\_\_() contains the logic to obtain the next element 
 * \_\_next\_\_() should raise StopIteration() when no elements are left

In [17]:
%reset -f 
class CapitalChars_Iterator(object):     
    def __init__(self,string): 
        self._string = string
        self._index = -1
    def __next__(self): 
        self._index += 1
        if self._index >= len(self._string): 
            raise StopIteration()
        if self._string[self._index].isupper():            
            return self._index
        # else returns None        
    def __iter__(self):        
        return self

print([i for i in CapitalChars_Iterator('SiSsA') if i != None])

[0, 2, 4]


## Unit testing framework

Why test? 
* testing is a crucial aspects of software development
 * it ensures code works as the developer thinks
 * it ensures code still works after changes 
 * in teamwork, it helps planning, understanding and sharing requirements
 
* tests should be devised before the software development
* python has a built-in test library: unittest

Workflow

* 0) import the module to be tested
* 1) define your class derived from unittest: the TestCase
* 2) fill it with test functions whose names start with 'test_'
* 3) run tests using unittest.main(\[argv,exit\]): argv and SystemExit in case an exception is raised





## TestCase methods to run the test

* setUpModule(), tearDownModule(): called at first and at last in module, run once 
* setUpClass(), tearDownClass: called at first and at last in class, run once, must be decorated @classmethod 
* setUp(), tearDown(): execute before/after any TestCase method
 



In [30]:
%reset -f
import unittest

class Err(object): pass

def setUpModule(): print("setUpModule")
def tearDownModule(): print("tearDownModule")

class Example1(unittest.TestCase):
    @classmethod
    def setUpClass(self): print(" setUpClass Example1")
    def setUp(self): print("  setUp")
    def test1(self): print("   test1")        
    def test2(self): 
        print("   test2")
        raise Err('error')
    def tearDown(self): print("  tearDown")
    @classmethod
    def tearDownClass(self): print(" tearDownClass Example1")

class Example2(unittest.TestCase): 
    @classmethod
    def setUpClass(cls): print(" setUpClass Example2")
    @classmethod
    def tearDownClass(cls): print(" tearDownClass Example2")
    def test3(self): print("   test3")        
    
print(unittest.main(argv=['first-arg-is-ignored'], exit=False))

.E.

setUpModule
 setUpClass Example1
  setUp
   test1
  tearDown
  setUp
   test2
  tearDown
 tearDownClass Example1
 setUpClass Example2
   test3
 tearDownClass Example2
tearDownModule



ERROR: test2 (__main__.Example1)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-30-9b56c873b00a>", line 16, in test2
    raise Err('error')
TypeError: object() takes no parameters

----------------------------------------------------------------------
Ran 3 tests in 0.004s

FAILED (errors=1)


## TestCase assert methods

Most common asserts 
 * assertFalse(x\[,msg\]), assertTrue(x\[,msg\]) 
 * assertEqual(a,b\[,msg\]), assertNotEqual(a,b\[,msg\]),assertGreater(a,b\[,msg\]), assertGreaterEqual(a,b\[,msg\]),assertLess(a,b\[,msg\]), assertLessEqual(a,b\[,msg\])  
 * assertIs(a,b\[,msg\]), assertIsNot(a,b\[,msg\]), assertIsNone(x\[,msg\]), assertIsNotNone(x\[,msg\]), 
 * assertIn(a,b\[,msg\]), assertNotIn(a,b\[,msg\])
 * assertIsInstance(a,b\[,msg\]), assertNotIsInstance(a,b\[,msg\])

Type-specific asserts
 * assertMultiLineEqual(a,b\[,msg\]), assertSequenceEqual(a,b\[,msg\]), assertListEqual(a,b\[,msg\]), assertTupleEqual(a,b\[,msg\]), assertSetEqual(a,b\[,msg\]), assertDictEqual(a,b\[,msg\])



It is possible to check if Exceptions, Warnings or Logs were invoked
* assertRaises(exc, fun, \*args, \*\*kwds) 
 * fun(\*args,\*\*kwds) raises exception exc
* assertRaisesRegex(exc, fun, \*args, \*\*kwds) 
 * fun(\*args,\*\*kwds) raises exception exc and message matches regex r
* assertWarns(warn, fun, \*args, \*\*kwds) 
 * fun(\*args,\*\*kwds) raises warning warn
* assertWarnsRegex()
 * fun(\*args,\*\*kwds) raises warning warn and message matches regex r
* assertLogs(logger,level)  -- TODO
 * the with block logs on logger with minimum level



In [65]:
%reset -f 
import unittest

def fun_raising_TypeError(): raise TypeError('type error')
def fun_raising_WarningMsg(): raise Warning('warning message')

class Test(unittest.TestCase):
    def test1(self): 
        self.assertRaises(TypeError,fun_raising_TypeError)
    def test2(self):
        self.assertRaises(Warning,fun_raising_WarningMsg)
    def test3(self): # should fail
        self.assertRaises(TypeError,fun_raising_WarningMsg)

print(unittest.main(argv=['first-arg-is-ignored'], exit=False))                        

..E


ERROR: test3 (__main__.Test)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-65-a467d41f55bd>", line 13, in test3
    self.assertRaises(TypeError,fun_raising_WarningMsg)
  File "/home/jaky/miniconda3/envs/py36/lib/python3.6/unittest/case.py", line 733, in assertRaises
    return context.handle('assertRaises', args, kwargs)
  File "/home/jaky/miniconda3/envs/py36/lib/python3.6/unittest/case.py", line 178, in handle
    callable_obj(*args, **kwargs)
  File "<ipython-input-65-a467d41f55bd>", line 5, in fun_raising_WarningMsg
    def fun_raising_WarningMsg(): raise Warning('warning message')

----------------------------------------------------------------------
Ran 3 tests in 0.015s

FAILED (errors=1)


## Skip decorators

Tests usefulness could be conditioned to conditions (e.g., platforms, libraries' version, ...)
* @unittest.skip(reason): skips the test
* @unittest.skipIf(condition, reason): skips if condition is True
* @unittest.skipUnless(condition, reason):  skips unless condition is True
* @unittest.expectedFailure: marks (but does not count) as failure 
* exception unittest.SkipTest(reason): to skip from inside the test 

In [678]:
%reset -f
import unittest

class Example(unittest.TestCase):
    trueCondition = True
    falseCondition = False
    @unittest.skipIf(trueCondition,\
                     "good reason to skip test1")
    def test1(self): pass
    @unittest.skipUnless(falseCondition,\
                         "good reason to skip test2")
    def test2(self): pass    
    def test3(self): 
        print("test3")
        if True: raise unittest.SkipTest("this makes test3 useless")
        print("not going to be printed")
    
print(unittest.main(argv=['first-arg-is-ignored'], exit=False))

sss

test3



----------------------------------------------------------------------
Ran 3 tests in 0.002s

OK (skipped=3)


## Iterate tests using subtests

* you can distinguish similar tests inside the body of the same test method using subTest
 * subTest(\[msg=None,\**params\]): test can be nested as context manager

In [6]:
%reset -f 
import unittest

global ListOfIntegers 
ListOfIntegers = [1,2,3,4,5,6,'7','8','9'] \
                            # '7' and
                            #     '8' and 
                            #         '9' are not integers

class Test(unittest.TestCase):
    def test_list(self): 
        for i in ListOfIntegers:
            with self.subTest(value=i):
                self.assertEqual(type(i), type(1))

print(unittest.main(argv=['first-arg-is-ignored'], exit=False))                


FAIL: test_list (__main__.Test) (value='7')
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-6-4c54590060b2>", line 13, in test_list
    self.assertEqual(type(i), type(1))
AssertionError: <class 'str'> != <class 'int'>

FAIL: test_list (__main__.Test) (value='8')
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-6-4c54590060b2>", line 13, in test_list
    self.assertEqual(type(i), type(1))
AssertionError: <class 'str'> != <class 'int'>

FAIL: test_list (__main__.Test) (value='9')
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-6-4c54590060b2>", line 13, in test_list
    self.assertEqual(type(i), type(1))
AssertionError: <class 'str'> != <class 'int'>

----------------------------------------------------------------------
Ran 1 test in 0.001s

FAILED

## Exercises